# Análisis de Posicionamiento de Precios

Este notebook analiza el posicionamiento de precios por canal, desde una vista general (familias) hasta el detalle SKU x canal.

Archivos requeridos en esta carpeta:
- `base.xlsx`: precios por SKU y canal (columnas: sku, descripción, UMV, Marca, Canal, Fuente, Precio, Peso neto UMV, Precio unitario, Fecha).
- `maestro_skus.xlsx`: maestro de SKUs (columnas: sku, categoria, familia).

Salidas esperadas: tablas resumen, KPIs (gap absoluto y porcentual, promedios, desviaciones) y gráficos útiles para detectar desviaciones relevantes.

In [ ]:
# Instalación automática (si falta algo). Ejecuta esta celda si obtienes errores de importación.
import importlib, sys, subprocess
def ensure(pkg):
    try:
        importlib.import_module(pkg)
    except ImportError:
        print(f'Instalando {pkg} ...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

for p in ['pandas','numpy','openpyxl','matplotlib','seaborn','plotly']:
    ensure(p)
print('Dependencias listas.')

In [ ]:
# Imports y configuración de entorno
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import unicodedata, re

pd.set_option('display.max_colwidth', 120)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
sns.set(style='whitegrid')

# Funciones utilitarias
def normalize_text(s):
    if s is None:
        return s
    s = str(s)
    s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    s = s.strip().lower()
    s = re.sub(r'\s+', '_', s)
    s = s.replace('%','pct')
    s = re.sub(r'[^a-z0-9_]+', '', s)
    return s

def to_numeric_safe(series):
    if series.dtype.kind in 'biufc':
        return series
    return pd.to_numeric(series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False), errors='coerce')

BASE_FILE = Path('base.xlsx')
MAESTRO_FILE = Path('maestro_skus.xlsx')
CANAL_INTERNO = 'interno'  # nombre exacto del canal interno en la columna 'Canal'
USAR_ULTIMA_FECHA = True   # si True, usa el último precio por SKU-Canal

BASE_FILE, MAESTRO_FILE

## Carga de datos

In [ ]:
# Leer base de precios
assert BASE_FILE.exists(), f'No se encontró {BASE_FILE.resolve()}'
try:
    df = pd.read_excel(BASE_FILE, sheet_name=0)
except Exception:
    df = pd.read_excel(BASE_FILE, sheet_name='base')

# Normalizar columnas
df.columns = [normalize_text(c) for c in df.columns]
df.rename(columns={
    'descripcion': 'descripcion',
    'peso_neto_umv': 'peso_neto_umv',
    'precio_unitario': 'precio_unitario',
    'fecha': 'fecha'
}, inplace=True)

# Tipos
for col in ['precio', 'precio_unitario', 'peso_neto_umv']:
    if col in df.columns:
        df[col] = to_numeric_safe(df[col])

if 'fecha' in df.columns:
    df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Normalizar campo canal
if 'canal' in df.columns:
    df['canal'] = df['canal'].astype(str).str.strip().str.lower()

# Definir precio de análisis: usa precio_unitario si existe; si no, precio/UMV si se puede, si no, precio
if 'precio_unitario' in df.columns and df['precio_unitario'].notna().any():
    precio_analisis = df['precio_unitario']
else:
    if 'precio' in df.columns and 'peso_neto_umv' in df.columns:
        precio_analisis = df['precio'] / df['peso_neto_umv'].replace(0, np.nan)
    else:
        precio_analisis = df['precio'] if 'precio' in df.columns else np.nan
df['precio_analisis'] = to_numeric_safe(precio_analisis)

print('Filas en base:', len(df))
display(df.head())
df.info()

In [ ]:
# Leer maestro de SKUs
assert MAESTRO_FILE.exists(), f'No se encontró {MAESTRO_FILE.resolve()}'
maestro = pd.read_excel(MAESTRO_FILE, sheet_name=0)
maestro.columns = [normalize_text(c) for c in maestro.columns]
keep = [c for c in ['sku','categoria','familia'] if c in maestro.columns]
maestro = maestro[keep].drop_duplicates()
maestro['sku'] = maestro['sku'].astype(str).str.strip()
print('Filas en maestro:', len(maestro))
display(maestro.head())

## Limpieza y selección de último precio por SKU-Canal

In [ ]:
# Asegurar tipos y claves de unión
df['sku'] = df['sku'].astype(str).str.strip()
if USAR_ULTIMA_FECHA and 'fecha' in df.columns:
    df_sorted = df.sort_values(['sku','canal','fecha'])
    df_latest = df_sorted.drop_duplicates(subset=['sku','canal'], keep='last')
else:
    df_latest = df.copy()

# Merge con maestro
data = df_latest.merge(maestro, on='sku', how='left')
cov = data['familia'].notna().mean() if 'familia' in data.columns else np.nan
print('Cobertura de maestro en data:', f'{cov:.1%}')
display(data.head())

# Diagnóstico general
print('Rango de fechas:', data['fecha'].min(), '→', data['fecha'].max())
display(data['canal'].value_counts().rename('filas_por_canal').to_frame())
display(data.groupby('canal')['precio_analisis'].agg(['count','mean','median','std']).sort_values('mean', ascending=False))

## Comparación canal interno vs resto (a nivel SKU)

In [ ]:
# Prepara precios internos y externos por SKU
interno = (data.query('canal == @CANAL_INTERNO')[['sku','precio_analisis']]
           .groupby('sku', as_index=False)['precio_analisis'].median()
          ).rename(columns={'precio_analisis':'precio_interno'})
externo = (data.query('canal != @CANAL_INTERNO')[['sku','precio_analisis','canal']]
           .groupby('sku', as_index=False)
           .agg(precio_externo=('precio_analisis','mean'), canales_externos=('canal','nunique'))
          )

sku_comp = maestro[['sku','categoria','familia']].merge(interno, on='sku', how='left').merge(externo, on='sku', how='left')
sku_comp['gap_abs'] = sku_comp['precio_externo'] - sku_comp['precio_interno']
sku_comp['gap_pct'] = np.where(sku_comp['precio_interno']>0, sku_comp['precio_externo']/sku_comp['precio_interno'] - 1, np.nan)

# Estadísticas por SKU entre canales
stats_por_sku = (data.groupby(['sku'])['precio_analisis']
                  .agg(media_precio=('mean'), std_precio=('std'), min_precio=('min'), max_precio=('max'), conteo=('count'))
                 )
sku_comp = sku_comp.merge(stats_por_sku, on='sku', how='left')

print('SKUs con comparables (interno y al menos un externo):',
      ((sku_comp['precio_interno'].notna()) & (sku_comp['precio_externo'].notna())).sum())
display(sku_comp.head(10))

## Top desviaciones por SKU

In [ ]:
sku_ok = sku_comp.dropna(subset=['precio_interno','precio_externo']).copy()
top_pos = sku_ok.sort_values('gap_abs', ascending=False).head(20)
top_neg = sku_ok.sort_values('gap_abs', ascending=True).head(20)
display(top_pos[['sku','familia','precio_interno','precio_externo','gap_abs','gap_pct','canales_externos']])
display(top_neg[['sku','familia','precio_interno','precio_externo','gap_abs','gap_pct','canales_externos']])

fig1 = px.bar(top_pos, x='sku', y='gap_abs', color='familia', title='Top 20 SKU con mayor gap positivo (externo > interno)')
fig1.update_layout(xaxis_title='SKU', yaxis_title='Gap absoluto')
fig1.show()

fig2 = px.bar(top_neg, x='sku', y='gap_abs', color='familia', title='Top 20 SKU con mayor gap negativo (interno > externo)')
fig2.update_layout(xaxis_title='SKU', yaxis_title='Gap absoluto')
fig2.show()

fig3 = px.histogram(sku_ok, x='gap_pct', nbins=50, title='Distribución gap porcentual (externo vs interno)')
fig3.update_layout(xaxis_title='Gap %', yaxis_title='Frecuencia')
fig3.show()

## Resumen por Familia (interno vs resto)

In [ ]:
fam = (sku_ok.groupby('familia')
       .agg(
           skus=('sku','nunique'),
           precio_interno_prom=('precio_interno','mean'),
           precio_externo_prom=('precio_externo','mean'),
           gap_abs_prom=('gap_abs','mean'),
           gap_abs_med=('gap_abs','median'),
           gap_pct_prom=('gap_pct','mean'),
           gap_pct_med=('gap_pct','median')
       )
      )
fam['gap_abs_total'] = fam['precio_externo_prom'] - fam['precio_interno_prom']
display(fam.sort_values('gap_abs_prom', ascending=False).head(20))

fig4 = px.bar(fam.reset_index().sort_values('gap_abs_prom', ascending=False).head(30),
              x='familia', y='gap_abs_prom', title='Gap absoluto promedio por familia (Top 30)')
fig4.update_layout(xaxis_title='Familia', yaxis_title='Gap absoluto prom.')
fig4.show()

fig5 = px.box(sku_ok, x='familia', y='gap_pct', points='outliers', title='Distribución gap % por familia')
fig5.update_layout(xaxis={'categoryorder':'total descending'}, yaxis_title='Gap %')
fig5.show()

## Comparación por Canal vs Interno (rátios)

In [ ]:
# Pivot SKU x Canal de precios
pivot = data.pivot_table(index='sku', columns='canal', values='precio_analisis', aggfunc='median')
if CANAL_INTERNO not in pivot.columns:
    print(f'Advertencia: no hay columna de canal interno = {CANAL_INTERNO!r} en los datos pivot.')

ratios = {}
for canal in pivot.columns:
    if canal == CANAL_INTERNO:
        continue
    ratios[canal] = (pivot[canal] / pivot[CANAL_INTERNO])

ratios_df = pd.DataFrame(ratios)
res_canal = pd.DataFrame({
    'ratio_promedio_vs_interno': ratios_df.mean(skipna=True),
    'ratio_mediana_vs_interno': ratios_df.median(skipna=True),
    'observaciones': ratios_df.count()
}).sort_values('ratio_promedio_vs_interno', ascending=False)
display(res_canal)

fig6 = px.bar(res_canal.reset_index(), x='index', y='ratio_promedio_vs_interno', title='Promedio de (precio canal / precio interno) por canal')
fig6.update_layout(xaxis_title='Canal', yaxis_title='Ratio promedio vs interno')
fig6.show()

## Heatmap: ratio por Familia y Canal (vs interno)

In [ ]:
# Calcular ratio por SKU-Canal vs interno y luego promediar por familia
base_ratios = data.merge(interno.rename(columns={'precio_interno':'precio_interno_ref'}), on='sku', how='left')
base_ratios['ratio_vs_interno'] = base_ratios['precio_analisis'] / base_ratios['precio_interno_ref']
fam_canal = (base_ratios[base_ratios['canal'] != CANAL_INTERNO]
             .groupby(['familia','canal'])['ratio_vs_interno']
             .mean().unstack('canal'))
plt.figure(figsize=(12, max(4, len(fam_canal)*0.4)))
sns.heatmap(fam_canal, annot=False, cmap='coolwarm', center=1.0)
plt.title('Ratio promedio (precio canal / interno) por Familia y Canal')
plt.xlabel('Canal')
plt.ylabel('Familia')
plt.tight_layout()
plt.show()

## Exportar resultados a Excel (opcional)

In [ ]:
# Guardar tablas clave en 'salidas/analisis_posicionamiento.xlsx'
out_dir = Path('salidas')
out_dir.mkdir(parents=True, exist_ok=True)
OUT_XLSX = out_dir / 'analisis_posicionamiento.xlsx'
with pd.ExcelWriter(OUT_XLSX, engine='openpyxl') as wb:
    sku_ok.to_excel(wb, sheet_name='sku_comp', index=False)
    fam.sort_values('gap_abs_prom', ascending=False).to_excel(wb, sheet_name='familia_resumen')
    res_canal.to_excel(wb, sheet_name='canal_vs_interno')
print('Exportado:', OUT_XLSX.resolve())